#transformers model and huggingface

In [ ]:
from transformers import BartTokenizer, BartForConditionalGeneration

In [ ]:
# Load model and tokenizer
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')


In [ ]:
model.to('cuda')

In [ ]:
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import math

class TextSummarizationDataset(Dataset):
    def __init__(self, dataframe):
        self.dataframe = dataframe

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        row = self.dataframe.iloc[idx]
        text = "summarize: " + row['Description']
        title = row['Title']
        return text, title



def create_data_loader(data_frame, batch_size=256):
    dataset = TextSummarizationDataset(data_frame)
    return DataLoader(dataset, batch_size=batch_size, shuffle=True)


In [ ]:
data_loader = create_data_loader(data_frame_cleaned, 512)

for texts, titles in data_loader:
    # Compute the lengths for each description in the batch
    lengths = [len(text.split()) for text in texts]
    max_len = max(lengths)
    min_len = max(min(lengths), 500)  # Ensure min_len is at least 500

    # Encode the text
    inputs = tokenizer(texts, return_tensors="pt", max_length=max_len, padding="longest", truncation=True)
    input_x = inputs['input_ids'].cuda()

    # Generate summaries
    summary_ids = model.generate(input_x, max_length=max_len, min_length=min_len, length_penalty=2.0, num_beams=4, early_stopping=True)
    summaries = [tokenizer.decode(g, skip_special_tokens=True) for g in summary_ids]

    # Save the output
    output_dict['name'].extend(titles)
    output_dict['summary'].extend(summaries)

    # Save all at once or incrementally
    temp_output = pd.DataFrame(output_dict)
  temp_output.to_csv(output_path, mode='a', header=not os.path.exists(output_path))  # Append mode



torch.Size([512, 5353])


In [ ]:
def add_summarize(text):
  text=str("summarize: " + text)
  return text

batch_size=4
data_numbers=len(data_frame_cleaned)
process_num=math.ceil(data_numbers/batch_size)


output_path='/content/drive/MyDrive/output_path.csv'
output_dict={'name':[],'summary':[]}


for i in range(process_num):
  #put data in a batch
  texts=list(data_frame_cleaned.iloc[batch_size*i:(i+1)*batch_size]['Description'].apply(add_summarize))

  #set min length of summary
  maxs=max(list(data_frame_cleaned.iloc[batch_size*i:(i+1)*batch_size]['len_of_Description']))
  mins=min(list(data_frame_cleaned.iloc[batch_size*i:(i+1)*batch_size]['len_of_Description']))
  min_len=min(500,int(0.5*mins))


  # Encode the text and generate summary
  inputs = tokenizer(texts, return_tensors="pt", max_length=maxs, padding="longest", truncation=True)
  #inputs = {k: v.cuda() for k, v in inputs.items()}




  # # Generate summaries
  summary_ids = model.generate(**inputs, max_length=maxs, min_length=min_len, length_penalty=2.0, num_beams=4, early_stopping=True)
  summaries = [tokenizer.decode(g, skip_special_tokens=True) for g in summary_ids]

  #save the output
  output_dict['name'].extend(list(data_frame_cleaned.iloc[batch_size*i:(i+1)*batch_size]['Title']))
  output_dict['summary'].extend(summaries)
  temp_output=pd.DataFrame(output_dict)
  temp_output.to_csv(output_path)




In [ ]:
import pandas as pd
#read .txt file and convert it to dataframe
file_path='/content/drive/MyDrive/booksummaries.txt'
fields=['ID', 'Tag', 'Title', 'Author', 'Year', 'Genre', 'Description']
data_frame = pd.read_csv(file_path, sep='\t', header=None, names=fields)
#cleaning data frame from missing value:
data_frame_cleaned = data_frame.dropna(axis=1)
#remove some Description that has small number of characters and their Description is missing
data_frame_cleaned['len_of_Description']=data_frame_cleaned['Description'].apply(len)
data_frame_cleaned=data_frame_cleaned[data_frame_cleaned['len_of_Description']>100]
data_frame_cleaned=data_frame_cleaned.sort_values(by=['len_of_Description'], ascending=True)

In [ ]:
inputs=inputs.cuda()
inputs.is_cuda

In [ ]:
summary_ids = model.generate(inputs.cuda(), max_length=1024, min_length=min_len, length_penalty=2.0, num_beams=4, early_stopping=True)
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

print(summary)

#preprocess

In [ ]:
!pip install langdetect

In [ ]:
from langdetect import detect, DetectorFactory
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
nltk.download('punkt')
import langdetect

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
# Set seed to make results reproducible
DetectorFactory.seed = 0

def detect_language(text):
    try:
        return langdetect.detect(text)
    except langdetect.lang_detect_exception.LangDetectException:
        return None

def find_non_english_sentences(text):
    sentences = sent_tokenize(text)
    non_english_sentences = [sentence for sentence in sentences if detect_language(sentence) != 'en']
    return non_english_sentences




def find_non_english_words(text):
    words = word_tokenize(text)
    non_english_words = [word for word in words if detect_language(word) != 'en' and len(word) > 1]
    return non_english_words


# Example Usage
sample_text = data_frame_cleaned.iloc[16514]['Description']
non_english_sentences = find_non_english_sentences(sample_text)
non_english_words = find_non_english_words(sample_text)

print("Non-English Sentences found:", non_english_sentences)
print("Non-English Words found:", non_english_words)



In [ ]:
!pip install spacy-langdetect

In [ ]:
import re
import nltk
import string

def preprocess_text(text):

  #url
  url_pattern = r'https?://\S+|www\.\S+'
  text = re.sub(url_pattern, '', text)
  #special char
  text=re.sub(r'[^\w\s]','',text)

  return text

In [ ]:
temp=data_frame_cleaned[data_frame_cleaned['Description'].apply(lambda x: bool(re.search(r'[^\x00-\x7F]+', x)))]

#text2image

In [ ]:
import cv2
import math
import nltk
import numpy as np
nltk.download('punkt')
from nltk.tokenize import word_tokenize, sent_tokenize
import  matplotlib.pyplot as plt

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
def text2image(text='',word_per_line=20,pixel_per_word=200,h_padding=100,v_padding=150,img_path=''):

    words=word_tokenize(data)
    number_of_line=math.ceil(len(words)/word_per_line)

    raw_image=np.ones(((v_padding*(number_of_line+1),(pixel_per_word*word_per_line)+(1*h_padding))))*255

    font = cv2.FONT_HERSHEY_SIMPLEX
    fontScale = 2
    color = 0
    thickness = 2

    for i in range(number_of_line):
      temp_text=words[i*word_per_line]
      for x in words[(i*word_per_line)+1:(i+1)*word_per_line]:
         temp_text+=(' '+x)
      raw_image= cv2.putText(raw_image, temp_text, (h_padding,v_padding*(i+1)), font,fontScale, color, thickness, cv2.LINE_AA)

    cv2.imwrite(img_path,raw_image)



In [ ]:
 text2image(text=data,word_per_line=20,pixel_per_word=200,h_padding=100,v_padding=100,img_path='/content/img.png')

#EXTRACTIVE APPROACHES

In [ ]:
import pandas as pd
#read .txt file and convert it to dataframe
file_path='/content/drive/MyDrive/booksummaries.txt'
fields=['ID', 'Tag', 'Title', 'Author', 'Year', 'Genre', 'Description']
data_frame = pd.read_csv(file_path, sep='\t', header=None, names=fields)
#cleaning data frame from missing value:
data_frame_cleaned = data_frame.dropna(axis=1)
#remove some Description that has small number of characters and their Description is missing
data_frame_cleaned['len_of_Description']=data_frame_cleaned['Description'].apply(len)
data_frame_cleaned=data_frame_cleaned[data_frame_cleaned['len_of_Description']>250]
#sort Description baseed on len_of_Description parameter
#data_frame_cleaned=data_frame_cleaned.sort_values(by=['len_of_Description'], ascending=True)

<ipython-input-1-bcd71ae77517>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_frame_cleaned['len_of_Description']=data_frame_cleaned['Description'].apply(len)


In [ ]:
data_frame_cleaned.head()

,ID,Tag,Title,Description,len_of_Description
0,620,/m/0hhy,Animal Farm,"Old Major, the old boar on the Manor Farm, ca...",5812
1,843,/m/0k36,A Clockwork Orange,"Alex, a teenager living in near-future Englan...",6121
2,986,/m/0ldx,The Plague,The text of The Plague is divided into five p...,6642
3,1756,/m/0sww,An Enquiry Concerning Human Understanding,The argument of the Enquiry proceeds by a ser...,17134
4,2080,/m/0wkt,A Fire Upon the Deep,The novel posits that space around the Milky ...,4493


In [ ]:
data=data_frame_cleaned.iloc[15]['Description']
data

' The book of Job tells the story of an extremely righteous man named Job, who is very prosperous and has seven sons and three daughters. Constantly fearing that his sons may have sinned and "cursed God in their hearts", he habitually offers burnt offerings as a pardon for their sins. The "sons of God" and Satan (literally "the Adversary") present themselves to God, and God asks Satan his opinion on Job. Satan answers that Job is pious only because God has put a "wall around" him and "blessed" his favourite servant with prosperity, but if God were to stretch out his hand and strike everything that Job had, then he would surely curse God. God gives Satan permission to test Job\'s righteousness. All Job\'s possessions are destroyed: 500 yoke of oxen and 500 donkeys carried off by Sabeans; 7,000 sheep burned up by \'The fire of God which fell from the sky\'; 3,000 camels stolen by the Chaldeans; and the house of the firstborn destroyed by a mighty wind, killing Job\'s ten children. Still 

#TEXT SUMMARIZATION USING THE FREQUENCY METHOD

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
import numpy as np
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
stopwords1 = set(stopwords.words("english"))
words = word_tokenize(data)

In [ ]:
freqTable = {}
for word in words:
  word = word.lower()

  if word in stopwords1:
      print(word)
  elif word in freqTable:
    freqTable[word] += 1
  else:
    freqTable[word] = 1

In [ ]:
sentences = sent_tokenize(data)

In [ ]:
sentenceValue = {}
for sentence in sentences:
  for word, freq in freqTable.items():
    if word in sentence.lower():
      if sentence in sentenceValue:
          sentenceValue[sentence] += freq
      else:
          sentenceValue[sentence] = freq


sumValues = 0
for sentence in sentenceValue:
  sumValues += sentenceValue[sentence]
average = int(sumValues / len(sentenceValue))

In [ ]:
summary = ''
for sentence in sentences:
  if (sentence in sentenceValue) and(sentenceValue[sentence] > (1.2 * average)):
    summary += "" + sentence

print(summary)

#Sumy


In [ ]:
!pip install sumy

In [ ]:
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.text_rank import TextRankSummarizer

In [ ]:
# Creating text parser using tokenization
parser = PlaintextParser.from_string(data, Tokenizer("english"))

In [ ]:
# Summarize using sumy TextRank
summarizer = TextRankSummarizer()
summary = summarizer(parser.document, 12)

In [ ]:
text_summary = ""
for sentence in summary:
  text_summary += str(sentence)

print(text_summary)

The book of Job tells the story of an extremely righteous man named Job, who is very prosperous and has seven sons and three daughters.The "sons of God" and Satan (literally "the Adversary") present themselves to God, and God asks Satan his opinion on Job.Satan answers that Job is pious only because God has put a "wall around" him and "blessed" his favourite servant with prosperity, but if God were to stretch out his hand and strike everything that Job had, then he would surely curse God.All Job's possessions are destroyed: 500 yoke of oxen and 500 donkeys carried off by Sabeans; 7,000 sheep burned up by 'The fire of God which fell from the sky'; 3,000 camels stolen by the Chaldeans; and the house of the firstborn destroyed by a mighty wind, killing Job's ten children.Still Job does not curse God, but instead shaves his head, tears his clothes, and says, "Naked I came out of my mother's womb, and naked shall I return: Lord has given, and Lord has taken away; blessed be the name of Lord

#Lex Rank


In [ ]:
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lex_rank import LexRankSummarizer

In [ ]:
parser = PlaintextParser.from_string(data, Tokenizer("english"))
summarizer = LexRankSummarizer()
summary = summarizer(parser.document, 2)

In [ ]:
dp = []
for i in summary:
  lp = str(i)
  dp.append(lp)
  final_sentence = ' '.join(dp)

print(final_sentence)

They adopt Seven Commandments of Animal-ism, the most important of which is, "All animals are equal". The changed commandments are as follows, with the changes bolded: * 4 No animal shall sleep in a bed with sheets.


#Luhn

In [ ]:
from sumy.summarizers.luhn import LuhnSummarizer

In [ ]:
parser = PlaintextParser.from_string(data, Tokenizer("english"))
summarizer_luhn = LuhnSummarizer()
summary_1 = summarizer_luhn(parser.document, 2)

In [ ]:
dp = []
for i in summary_1:
  lp = str(i)
  dp.append(lp)
  final_sentence = ' '.join(dp)



print(final_sentence)

Later, Napoleon and his pigs secretly revise some commandments to clear them of accusations of law-breaking (such as "No animal shall drink alcohol" having "to excess" appended to it and "No animal shall sleep in a bed" with "with sheets" added to it). Eventually these are replaced with the maxims, "All animals are equal, but some animals are more equal than others", and "Four legs good, two legs better!"


#LSA

In [ ]:
from sumy.summarizers.lsa import LsaSummarizer

In [ ]:
parser = PlaintextParser.from_string(data, Tokenizer("english"))
summarizer_lsa = LsaSummarizer()
summary_2 = summarizer_lsa(parser.document, 2)

In [ ]:
for i in summary_2:
  lp = str(i)
  dp.append(lp)
  final_sentence = ' '.join(dp)

print(final_sentence)

Later, Napoleon and his pigs secretly revise some commandments to clear them of accusations of law-breaking (such as "No animal shall drink alcohol" having "to excess" appended to it and "No animal shall sleep in a bed" with "with sheets" added to it). Eventually these are replaced with the maxims, "All animals are equal, but some animals are more equal than others", and "Four legs good, two legs better!" The pigs elevate themselves to positions of leadership and set aside special food items, ostensibly for their personal health. The pigs Snowball, Napoleon, and Squealer adapt Old Major's ideas into an actual philosophy, which they formally name Animalism.


#score sentence based on cosine distance of sentence


In [ ]:
import numpy as np
import networkx as nx
import nltk, re
from nltk.corpus import stopwords
from nltk.cluster.util import cosine_distance
from nltk.tokenize import sent_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
def read_text(txt: str = ""):
  sentences = []
  sentences = sent_tokenize(txt)
  for sentence in sentences:
    sentence.replace("[^a-zA-Z0-9]", " ")
  return sentences



def sentence_similarity(sentence1, sentence2, stopwords = []):
  sentence1 = [word.lower() for word in sentence1]
  sentence2 = [word.lower() for word in sentence2]
  all_words = list(set(sentence1 + sentence2))
  vector1 = [0] * len(all_words)
  vector2 = [0] * len(all_words)
  # First sentence vector
  for word in sentence1:
    if not word in stopwords:
      vector1[all_words.index(word)] += 1
  # Second sentence vector
  for word in sentence2:
    if not word in stopwords:
      vector2[all_words.index(word)] += 1
  # Vectors cosine similarity
  return 1 - cosine_distance(vector1, vector2)


def sentences_similarity_matrix(sentences, stopwords_):
  similarity_matrix = np.zeros((len(sentences), len(sentences))) # N on N
  for i in range(len(sentences)):
      for j in range(len(sentences)):
        if i != j:
          similarity_matrix[i][j] = sentence_similarity(sentences[i], sentences[j], stopwords_)
  return similarity_matrix

In [ ]:

def summarize(txt, top_n):


  nltk.download('stopwords')
  nltk.download('punkt')
  final_stopwords = stopwords.words('english')



  summarized_text = []
  # Read and tokenize txt
  sentences = read_text(txt)

  # Get similarity matrix
  sentence_similarity_matrix = sentences_similarity_matrix(sentences, final_stopwords)
  # Rank sentences in the given similarity matrix
  sentence_similarity_graph = nx.from_numpy_array(sentence_similarity_matrix)
  scores = nx.pagerank(sentence_similarity_graph)
  # Sort the rank of top sentences
  ranked_sentences = sorted(((scores[i], s) for i, s in enumerate(sentences)), reverse = True)
  # Get the top n number of sentences based on rank
  for i in range(top_n):
     summarized_text.append(ranked_sentences[i][1])
  # Output the summarized version
  summary = " ".join(summarized_text)
  return summary, len(sentences)

#score sentence based on frequency

In [ ]:
import re # For extra cleaning
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import sent_tokenize,word_tokenize
from nltk.corpus import stopwords
import heapq

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
def clean_text(txt):
  text = re.sub(r"\[[0-9]*\]", " ", txt)
  text = text.lower()
  text = re.sub(r'\s+'," ", text)
  text = re.sub(r","," ", text)
  return text

In [ ]:
cleaned_text = clean_text(data)

In [ ]:
words_frequency = {}
word_tokens = word_tokenize(cleaned_text)
stopwords =  set(stopwords.words("english"))
for word in word_tokens:
    if word not in stopwords:
        if word not in words_frequency.keys():
            words_frequency[word]=1
        else:
            words_frequency[word] +=1
maximum_frequency = max(words_frequency.values())
for word in words_frequency.keys():
    words_frequency[word] = (words_frequency[word] / maximum_frequency)

In [ ]:
sentences_score = {}
sentences_tokens = sent_tokenize(cleaned_text)
for sentence in sentences_tokens:
    for word in word_tokenize(sentence):
        if word in words_frequency.keys():
            if (len(sentence.split(" "))) < 30:
                if sentence not in sentences_score.keys():
                    sentences_score[sentence] = words_frequency[word]
                else:
                    sentences_score[sentence] += words_frequency[word]

In [ ]:
def get_key(desired_value):
    for key, value in sentences_score.items():
        if desired_value == value:
            return key
key = get_key(max(sentences_score.values()))
print(max(sentences_score.values()),key)

8.666666666666666 the "sons of god" and satan (literally "the adversary") present themselves to god  and god asks satan his opinion on job.


In [ ]:
def get_key(desired_value):
    for key, value in sentences_score.items():
        if desired_value == value:
            return key
key = get_key(max(sentences_score.values()))
N = 5
summary = heapq.nlargest(N, sentences_score, key = sentences_score.get)
print(" ".join(summary))

the "sons of god" and satan (literally "the adversary") present themselves to god  and god asks satan his opinion on job. his wife prompts him to "curse god  and die " but job answers  "you speak as one of the foolish speaks.  the book of job tells the story of an extremely righteous man named job  who is very prosperous and has seven sons and three daughters. constantly fearing that his sons may have sinned and "cursed god in their hearts"  he habitually offers burnt offerings as a pardon for their sins. god then rebukes the three friends and says  "i am angry with you... you have not spoken of me what is right."


#Extractive Approach 3

In [ ]:
!pip install spacy
!python -m spacy download en_core_web_lg
!pip install pytextrank

In [ ]:
import spacy
import pytextrank

In [ ]:
nlp = spacy.load("en_core_web_lg")
nlp.add_pipe("textrank")

In [ ]:
doc = nlp(data)

In [ ]:
n_phrases, n_sentences = 2, 4
for sent in doc._.textrank.summary(limit_phrases = n_phrases, limit_sentences = n_sentences):
  print(sent)

#transforemer

In [ ]:
from transformers import PegasusForConditionalGeneration
from transformers import PegasusTokenizer
from transformers import pipeline

In [ ]:
len(data)

2635

In [ ]:
# Pick model
model_name = "google/pegasus-xsum"
# Load pretrained tokenizer
pegasus_tokenizer = PegasusTokenizer.from_pretrained(model_name)
# Define PEGASUS model
pegasus_model = PegasusForConditionalGeneration.from_pretrained(model_name)
# Create tokens
tokens = pegasus_tokenizer(data, truncation = True, padding = "longest", return_tensors = "pt")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/87.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.52M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/259 [00:00<?, ?B/s]

In [ ]:
# Summarize the desired text
encoded_summary = pegasus_model.generate(**tokens)
# Decode the summarized text
decoded_summary = pegasus_tokenizer.decode(encoded_summary[0], skip_special_tokens = True)

In [ ]:
decoded_summary

'In our series of letters from African journalists, film-maker and columnist Ahmed Rashid looks at one of the most famous curses in the Bible.'

In [ ]:
summarizer = pipeline("summarization", model = model_name, tokenizer = pegasus_tokenizer, framework = "pt")
# Create summary
summary = summarizer(data, min_length = 200, max_length = 800, truncation = True)
print(summary[0]["summary_text"])

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Your max_length is set to 800, but your input_length is only 512. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=256)


In our series of letters from African journalists, film-maker and columnist Farai Sevenzo looks at the story of Job, one of the most famous biblical stories of all time, and examines why it is so important for us to know that there is no such thing as a "curse" or "cursed day" when it comes to God and God's people, and why it is so important for us to know that there is no such thing as a "curse" or "cursed day" when it comes to God and God's people, and why it is so important for us to know that there is no such thing as a "curse" or "cursed day" when it comes to God and God's people, and why it is so important for us to know that there is no such thing as a "curse" or "cursed day" when it comes to God and God's people, and why it is so important for us to know that there is no such thing as a "curse" or "cursed day" when it comes to God and God's people, and why it is so important for us to know that there is no such thing as a "curse" or "cursed day" when it comes to God and God's p

In [ ]:
data

' The book of Job tells the story of an extremely righteous man named Job, who is very prosperous and has seven sons and three daughters. Constantly fearing that his sons may have sinned and "cursed God in their hearts", he habitually offers burnt offerings as a pardon for their sins. The "sons of God" and Satan (literally "the Adversary") present themselves to God, and God asks Satan his opinion on Job. Satan answers that Job is pious only because God has put a "wall around" him and "blessed" his favourite servant with prosperity, but if God were to stretch out his hand and strike everything that Job had, then he would surely curse God. God gives Satan permission to test Job\'s righteousness. All Job\'s possessions are destroyed: 500 yoke of oxen and 500 donkeys carried off by Sabeans; 7,000 sheep burned up by \'The fire of God which fell from the sky\'; 3,000 camels stolen by the Chaldeans; and the house of the firstborn destroyed by a mighty wind, killing Job\'s ten children. Still 